In [ ]:
#%%capture
%pip install accelerate peft bitsandbytes transformers trl
%pip install transformers
%pip install huggingface-hub

  Using cached huggingface_hub-0.22.2-py3-none-any.whl (388 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.14.1
    Uninstalling huggingface-hub-0.14.1:
      Successfully uninstalled huggingface-hub-0.14.1


In [ ]:
import os
import torch
import pandas as pd
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer
from datasets import Dataset

from peft import PeftModel

In [ ]:
from google.colab import userdata
from google.colab import drive
from sklearn.model_selection import train_test_split

In [ ]:
# use secret token
hf_token = userdata.get('hf-token')

# Model from Hugging Face hub
base_model = "meta-llama/Llama-2-7b-hf"

# New instruction dataset
ads500B_dataset = "current_questions.csv"

# Fine-tuned model
new_model = "llama-2-7b-chat-ads500b"

In [ ]:
!huggingface-cli login --token $hf_token

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
base_dir = '/content/drive/MyDrive/'

In [ ]:
def read_dataset(file_path):
    with open(file_path, 'r', encoding="cp1252") as file:
        lines = file.readlines()
    return lines

dataset = read_dataset(base_dir + ads500B_dataset)

In [ ]:
def show_set(showset):
  for s in showset:
    print(s)

show_set(dataset[0:5])

### Human: What are the key topics covered in the ADS 500B Data Science Programming course?, ### Assistant: The key topics covered in the ADS 500B course include software languages, exploratory data analysis, analytics, basic machine learning techniques, Unix, SQL, R, and Python.,

### Human: What types of questions are included in the quizzes for this course?, ### Assistant: The quizzes in this course consist of multiple choice and true and false questions.,

### Human: What is the focus of Module 1 in ADS 500B?, ### Assistant: Module 1 focuses on defining key terminology and concepts related to computational thinking, data collections, preparation, and processing techniques.,

### Human: Why is it recommended to install the full graphical user interface version of Anaconda for this course?, ### Assistant: Installing the full graphical user interface version of Anaconda is recommended for setting up the Python management environment used in the course.,

### Human: What is the importa

In [ ]:
for i, d in enumerate(dataset):
  d = d.replace(", ###", "\n###").strip()
  d = d[:-1]
  dataset[i] = d

show_set(dataset[0:5])

### Human: What are the key topics covered in the ADS 500B Data Science Programming course?
### Assistant: The key topics covered in the ADS 500B course include software languages, exploratory data analysis, analytics, basic machine learning techniques, Unix, SQL, R, and Python.
### Human: What types of questions are included in the quizzes for this course?
### Assistant: The quizzes in this course consist of multiple choice and true and false questions.
### Human: What is the focus of Module 1 in ADS 500B?
### Assistant: Module 1 focuses on defining key terminology and concepts related to computational thinking, data collections, preparation, and processing techniques.
### Human: Why is it recommended to install the full graphical user interface version of Anaconda for this course?
### Assistant: Installing the full graphical user interface version of Anaconda is recommended for setting up the Python management environment used in the course.
### Human: What is the importance of learn

In [ ]:
len(dataset)

1212

In [ ]:
def split_dataset(data, test_size=0.1):
    train_set, test_set = train_test_split(data, test_size=test_size, random_state=42)
    return train_set, test_set

In [ ]:
train_set, test_set = split_dataset(dataset)

In [ ]:
df = pd.DataFrame(train_set, columns=['text'])

In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    token=hf_token,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    trust_remote_code=True,
    token=hf_token)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [ ]:
dataset = Dataset.from_pandas(df)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1090 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss
25,1.724000
50,1.282900
75,1.262400
100,1.120800
125,1.202500
150,1.107500
175,1.136500
200,1.109500
225,1.205300
250,1.130300


Checkpoint destination directory ./results/checkpoint-25 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-50 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-75 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-100 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-125 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-150 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-175 already exists and is non-empty. Saving will proceed but saved results may 

TrainOutput(global_step=273, training_loss=1.2167556818588312, metrics={'train_runtime': 745.5495, 'train_samples_per_second': 1.462, 'train_steps_per_second': 0.366, 'total_flos': 2530979064053760.0, 'train_loss': 1.2167556818588312, 'epoch': 1.0})

In [ ]:
trainer.model.save_pretrained(base_dir+new_model)
trainer.tokenizer.save_pretrained(base_dir+new_model)

('/content/drive/MyDrive/llama-2-7b-chat-ads500b/tokenizer_config.json',
 '/content/drive/MyDrive/llama-2-7b-chat-ads500b/special_tokens_map.json',
 '/content/drive/MyDrive/llama-2-7b-chat-ads500b/tokenizer.model',
 '/content/drive/MyDrive/llama-2-7b-chat-ads500b/added_tokens.json',
 '/content/drive/MyDrive/llama-2-7b-chat-ads500b/tokenizer.json')

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
load_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    token=hf_token,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)



config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

NameError: name 'PeftModel' is not defined

In [ ]:
model = PeftModel.from_pretrained(load_model, base_dir+new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(base_model, token=hf_token, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
model.save_pretrained(base_dir+new_model+"norm")
tokenizer.save_pretrained(base_dir+new_model+"norm")

('/content/drive/MyDrive/llama-2-7b-chat-ads500bnorm/tokenizer_config.json',
 '/content/drive/MyDrive/llama-2-7b-chat-ads500bnorm/special_tokens_map.json',
 '/content/drive/MyDrive/llama-2-7b-chat-ads500bnorm/tokenizer.model',
 '/content/drive/MyDrive/llama-2-7b-chat-ads500bnorm/added_tokens.json',
 '/content/drive/MyDrive/llama-2-7b-chat-ads500bnorm/tokenizer.json')